In [295]:
import json
import numpy as np
import pandas as pd

import MeCab

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.manifold import TSNE

import plotly.express as px
from datetime import datetime


In [296]:
stories_json = []
with open('./output_stories_0123.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

839
dict_keys(['index', 'stories_index', 'title', 'broadcasting_date', 'abstracts_list'])


In [297]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママにた...
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、天...


In [298]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 品詞を絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

In [299]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_joined'].str.len() > 0] 
sentences = stories_df["abstract_joined"].apply(mecab_tokenizer)
print(sentences)

0      [何, のび太, 魔法, 魔法, 便利, ちがい, もしもボックス, 現実, 世界, 魔法,...
1      [朝, 寝坊, のび太, ドラえもん, 爆発, コショウ, コショウ, 行き先, そば, 人...
2      [みんな, 家族, お花見, 話, しかた, のび太, ママ, パパ, ゴルフ, 予定, の...
3      [スネ夫, ハワイ旅行, じまん, のび太, 自分, ハワイ, ママ, 給料日, 前, お金...
4      [学校, 天女, 羽衣, 羽子, ろ, 劇, げき, こと, のび太, しずか, 劇, 最後...
                             ...                        
834    [星がきれい, 夜, のび太, スネ夫, 家, 前, 庭, 星空, パーティー, 女の子, ...
835    [おなか, 食事, のび太, ご飯, ありがたみ, ママ, のび太, まじめ, 米, パン,...
836    [子供達, 鬼ごっこ, の, のび太, 自分たち, 野郎, みんな, ジャイアン, スネ夫,...
837    [５, 回, 連続, ０, 点, のび太, 来週, テスト, ０, 点, ママ, 鬼, 顔,...
838    [冬, スポーツ, ウインタースポーツ, ランド, のび太, たち, スキー, スケート, ...
Name: abstract_joined, Length: 839, dtype: object


In [300]:
abstract_list = sentences.tolist()
trainings = [TaggedDocument(words = data, tags = [i]) for i,data in enumerate(abstract_list)]
m = Doc2Vec(documents= trainings, dm = 1, window=8, min_count=10, workers=4)


In [301]:
vector_list = m.dv.vectors
tsne = TSNE(n_components=2)
vector_enbedded = tsne.fit_transform(vector_list)
print(vector_enbedded)

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[[-20.433289    4.7130165]
 [-20.310411    2.8583996]
 [ -6.518787   14.703737 ]
 ...
 [-33.856182    1.4472531]
 [-40.546482   -8.736391 ]
 [-30.979641   -0.6708592]]


In [302]:
vector_df = pd.DataFrame.from_dict({
    'vector': [v for v in vector_list],
    'vector_x': [v[0] for v in vector_enbedded],
    'vector_y': [v[1] for v in vector_enbedded]
})
stories_df_merged = stories_df.join(vector_df)
stories_df_merged['broadcasting_dt'] = stories_df_merged['broadcasting_date'].apply(lambda d: datetime.strptime(d, '%Y年%m月%d日'))
stories_df_merged['broadcasting_year'] = stories_df_merged['broadcasting_dt'].dt.year

In [303]:
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', color='broadcasting_year', hover_data=["index", "title"])
fig.show()

In [304]:
df = px.data.tips()
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', hover_data=["index", "title"], facet_row="broadcasting_year", width=800, height=2000)
fig.show()

In [305]:
sampled_df = stories_df_merged.sample(1)
print(sampled_df.head())
sampled_df = sampled_df.rename(columns={"vector": "sampled_vec"})[["sampled_vec"]]


   index  stories_index        title broadcasting_date  \
14  0332              1  「ジャイアンがパンダ」        2013年5月31日   

                                       abstracts_list  \
14  [　めずらしく、落ち込むジャイアンを見かけたのび太が気になって話を聞いたところ、三輪車に乗っ...   

                                      abstract_joined  \
14  　めずらしく、落ち込むジャイアンを見かけたのび太が気になって話を聞いたところ、三輪車に乗って...   

                                               vector   vector_x   vector_y  \
14  [-0.0004075789, 0.014004873, 0.0008815174, 0.0...  38.492756 -16.840635   

   broadcasting_dt  broadcasting_year  
14      2013-05-31               2013  


In [306]:
stories_df_merged = stories_df_merged.assign(key=1).merge(sampled_df.assign(key=1), on='key').drop('key', axis=1)
stories_df_merged

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined,vector,vector_x,vector_y,broadcasting_dt,broadcasting_year,sampled_vec
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...,"[-0.03860779, 0.023010895, -0.009547177, 0.013...",-20.433289,4.713017,2013-03-15,2013,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...,"[-0.03695579, 0.02311029, -0.0061565125, 0.006...",-20.310411,2.858400,2013-04-12,2013,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママにた...,"[-0.034850962, 0.013524959, 0.0120846685, -0.0...",-6.518787,14.703737,2013-04-12,2013,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...,"[-0.018790789, 0.01952498, -0.0012746033, 0.00...",15.450960,-0.729094,2013-04-26,2013,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、天...,"[-0.018666945, 0.0032326754, -0.0021730692, 0....",27.591757,-0.631972,2013-04-26,2013,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
834,0741,1,「夜空に輝くピザ・ギョーザ」,2023年1月14日,[星がきれいにかがやく夜、のび太がスネ夫の家の前を通ると、庭で星空パーティーが開かれていた。...,星がきれいにかがやく夜、のび太がスネ夫の家の前を通ると、庭で星空パーティーが開かれていた。女...,"[-0.042310946, 0.037225574, -0.008514948, 0.00...",-32.250496,6.015155,2023-01-14,2023,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
835,0742,0,「ありがたみわかり機」,2023年1月21日,[おなかがいっぱいだからと食事を残すのび太に、ご飯が食べられるありがたみについて話すママ。と...,おなかがいっぱいだからと食事を残すのび太に、ご飯が食べられるありがたみについて話すママ。とこ...,"[-0.005700744, 0.002782856, 0.0021310926, 0.00...",33.463036,-8.746819,2023-01-21,2023,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
836,0742,1,「のび泥棒をタイホせよ！」,2023年1月21日,[小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろうとみんな...,小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろうとみんなを...,"[-0.049013015, 0.03845185, -0.0042034015, 0.01...",-33.856182,1.447253,2023-01-21,2023,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
837,0743,0,「鬼のパンツはいいパンツ？」,2023年1月28日,[５回連続で０点を取ってしまったのび太。来週のテストも０点だったら、ママが鬼（おに）になって...,５回連続で０点を取ってしまったのび太。来週のテストも０点だったら、ママが鬼（おに）になってし...,"[-0.02664007, 0.025795339, -0.0013815146, -0.0...",-40.546482,-8.736391,2023-01-28,2023,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."


In [307]:
stories_df_merged

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined,vector,vector_x,vector_y,broadcasting_dt,broadcasting_year,sampled_vec
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...,"[-0.03860779, 0.023010895, -0.009547177, 0.013...",-20.433289,4.713017,2013-03-15,2013,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...,"[-0.03695579, 0.02311029, -0.0061565125, 0.006...",-20.310411,2.858400,2013-04-12,2013,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママにた...,"[-0.034850962, 0.013524959, 0.0120846685, -0.0...",-6.518787,14.703737,2013-04-12,2013,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...,"[-0.018790789, 0.01952498, -0.0012746033, 0.00...",15.450960,-0.729094,2013-04-26,2013,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、天...,"[-0.018666945, 0.0032326754, -0.0021730692, 0....",27.591757,-0.631972,2013-04-26,2013,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
834,0741,1,「夜空に輝くピザ・ギョーザ」,2023年1月14日,[星がきれいにかがやく夜、のび太がスネ夫の家の前を通ると、庭で星空パーティーが開かれていた。...,星がきれいにかがやく夜、のび太がスネ夫の家の前を通ると、庭で星空パーティーが開かれていた。女...,"[-0.042310946, 0.037225574, -0.008514948, 0.00...",-32.250496,6.015155,2023-01-14,2023,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
835,0742,0,「ありがたみわかり機」,2023年1月21日,[おなかがいっぱいだからと食事を残すのび太に、ご飯が食べられるありがたみについて話すママ。と...,おなかがいっぱいだからと食事を残すのび太に、ご飯が食べられるありがたみについて話すママ。とこ...,"[-0.005700744, 0.002782856, 0.0021310926, 0.00...",33.463036,-8.746819,2023-01-21,2023,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
836,0742,1,「のび泥棒をタイホせよ！」,2023年1月21日,[小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろうとみんな...,小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろうとみんなを...,"[-0.049013015, 0.03845185, -0.0042034015, 0.01...",-33.856182,1.447253,2023-01-21,2023,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."
837,0743,0,「鬼のパンツはいいパンツ？」,2023年1月28日,[５回連続で０点を取ってしまったのび太。来週のテストも０点だったら、ママが鬼（おに）になって...,５回連続で０点を取ってしまったのび太。来週のテストも０点だったら、ママが鬼（おに）になってし...,"[-0.02664007, 0.025795339, -0.0013815146, -0.0...",-40.546482,-8.736391,2023-01-28,2023,"[-0.0004075789, 0.014004873, 0.0008815174, 0.0..."


In [308]:
from scipy.spatial.distance import cosine

stories_df_merged["cosine_similarity"] =  stories_df_merged.apply(lambda v: 1 - cosine(v["vector"], v["sampled_vec"]), axis=1)
stories_df_merged_sort = stories_df_merged[["title", "abstract_joined", "cosine_similarity"]].sort_values("cosine_similarity", ascending=False)


In [309]:
stories_df_merged_sort

,title,abstract_joined,cosine_similarity
14,「ジャイアンがパンダ」,めずらしく、落ち込むジャイアンを見かけたのび太が気になって話を聞いたところ、三輪車に乗って...,1.000000
95,「怪談ランプ」,\r\n ジャイアンたちと怪談（かいだん）大会をやることになったのび太は、こわい話をしてジャ...,0.824409
697,「落としものカムバックスプレー」,買ったばかりのマンガをジャイアンにうばわれ、悲しみにくれるのび太。しかも、部屋にもどると、な...,0.808314
148,「ジャイアンが飛んできた」,スネ夫の家の庭でテニスをしていたのび太は、空振りして転んでしまい、しずかの前でスネ夫に大笑...,0.801245
604,「誕生！？大統領のび太」,ママのお手伝いをしておこづかいをもらおうとたくらむのび太だったが、ママに見すかされてしまい、...,0.801086
...,...,...,...
823,「リフトストック」,部屋でステッキのようなものを手入れするドラえもんに、それは何かとたずねるのび太。ドラえもんは...,-0.689870
90,「『真実の旗印』はつねに正しい」,\r\n 宿題をわすれたのび太は、ウソの言い訳をして先生から怒（おこ）られ、帰りがおそくなっ...,-0.693743
151,「ニクメナイン」,\r\n 空き地で野球の練習をしていたのび太たち。いつもに増してキゲンの悪そうなジャイアンに...,-0.708073
149,「わすれろ草」,空き地にジャイアンとスネ夫の似顔絵を落書きし、みんなに見せびらかしていたのび太。するとそこ...,-0.717363
